In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

In [12]:
import nltk

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [16]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...


True

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

In [7]:
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

376 documents
41 classes ['adverse_drug', 'agen_right', 'agent_acquaintance', 'agent_age', 'agent_annoying', 'agent_bad', 'agent_beautiful', 'agent_beclever', 'agent_birthday', 'agent_boring', 'agent_boss', 'agent_busy', 'agent_canyouhelp', 'agent_chatbot', 'agent_clever', 'agent_crazy', 'agent_fire', 'agent_funny', 'agent_good', 'agent_happy', 'agent_hobby', 'agent_hungry', 'agent_marryuser', 'agent_myfriend', 'agent_occupation', 'agent_origin', 'agent_ready', 'agent_real', 'agent_religion', 'agent_residence', 'agent_sure', 'agent_talktome', 'agent_there', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
264 unique lemmatized words ["'s", ',', '[', ']', 'a', 'adalah', 'adverse', 'agama', 'akan', 'aku', 'all', 'anda', 'aneh', 'anyone', 'apa', 'apakah', 'are', 'asal|negara', 'awesome', 'bagaimana', 'bahagia', 'baik', 'bantu', 'bantuanmu', 'banyak', 'be', 'behavior', 'bekerja', 'bekerja|sibuk|bekerja', 'belajar',

In [9]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\user\AppData\Local\Temp\ipykernel_15300\2836083368.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [10]:
import tensorflow as tf
from tensorflow.keras.optimizers import SGD

In [11]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1e-6,
    decay_rate=0.9)
optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)

In [12]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
# sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [13]:
#fitting the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
76/76 [==============================] - 1s 1ms/step - loss: 3.7205 - accuracy: 0.0293
Epoch 2/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7245 - accuracy: 0.0213
Epoch 3/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7158 - accuracy: 0.0266
Epoch 4/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7218 - accuracy: 0.0186
Epoch 5/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7116 - accuracy: 0.0372
Epoch 6/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7183 - accuracy: 0.0319
Epoch 7/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7173 - accuracy: 0.0160
Epoch 8/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7201 - accuracy: 0.0266
Epoch 9/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7141 - accuracy: 0.0186
Epoch 10/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7117 - accuracy: 0.0186
Epoch 11/

76/76 [==============================] - 0s 1ms/step - loss: 3.7131 - accuracy: 0.0319
Epoch 84/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7175 - accuracy: 0.0239
Epoch 85/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7156 - accuracy: 0.0293
Epoch 86/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7137 - accuracy: 0.0186
Epoch 87/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7165 - accuracy: 0.0239
Epoch 88/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7168 - accuracy: 0.0346
Epoch 89/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7254 - accuracy: 0.0239
Epoch 90/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7154 - accuracy: 0.0293
Epoch 91/200
76/76 [==============================] - 0s 962us/step - loss: 3.7172 - accuracy: 0.0266
Epoch 92/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7176 - accuracy: 0.0266
Epoch 93/20

76/76 [==============================] - 0s 1ms/step - loss: 3.7208 - accuracy: 0.0186
Epoch 165/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7210 - accuracy: 0.0239
Epoch 166/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7141 - accuracy: 0.0266
Epoch 167/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7153 - accuracy: 0.0239
Epoch 168/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7132 - accuracy: 0.0160
Epoch 169/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7169 - accuracy: 0.0213
Epoch 170/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7157 - accuracy: 0.0293
Epoch 171/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7145 - accuracy: 0.0346
Epoch 172/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7119 - accuracy: 0.0293
Epoch 173/200
76/76 [==============================] - 0s 1ms/step - loss: 3.7198 - accuracy: 0.0426
Epoc

In [13]:
# Saving the model
model.save('chatbot_model.h5', hist)
print("model created")

model created


In [43]:
import tensorflow as tf
from transformers import TFBertForQuestionAnswering, BertTokenizer

In [34]:
pip install transformers

     ---------------------------------------- 7.1/7.1 MB 4.9 MB/s eta 0:00:00
     -------------------------------------- 224.5/224.5 kB 6.9 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 13.1 MB/s eta 0:00:00
     -------------------------------------- 151.6/151.6 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.1 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
chatterbot-corpus 1.2.0 requires PyYAML<4.0,>=3.12, but you have pyyaml 6.0 which is incompatible.


In [45]:


# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForQuestionAnswering.from_pretrained(model_name)

# Encode input question and context
question = "What is the capital of France?"
context = "Paris is the capital of France."
encoded_inputs = tokenizer(question, context, return_tensors="tf")

# Answer the question
start_scores, end_scores = model(encoded_inputs["input_ids"], attention_mask=encoded_inputs["attention_mask"])

# Find the start and end indices of the answer in the context
start_index = tf.argmax(start_scores, axis=1).numpy()[0]
end_index = tf.argmax(end_scores, axis=1).numpy()[0]

# Decode the answer
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(encoded_inputs["input_ids"][0][start_index:end_index+1]))

print("Question:", question)
print("Answer:", answer)


All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


InvalidArgumentError: Value for attr 'T' of string is not in the list of allowed values: float, double, int32, uint8, int16, int8, complex64, int64, qint8, quint8, qint32, bfloat16, uint16, complex128, half, uint32, uint64, bool
	; NodeDef: {{node ArgMax}}; Op<name=ArgMax; signature=input:T, dimension:Tidx -> output:output_type; attr=T:type,allowed=[DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, 16534343205130372495, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64, DT_BOOL]; attr=Tidx:type,default=DT_INT32,allowed=[DT_INT16, DT_INT32, DT_INT64]; attr=output_type:type,default=DT_INT64,allowed=[DT_INT16, DT_UINT16, DT_INT32, DT_INT64]> [Op:ArgMax]

In [47]:
from transformers import TFBertForQuestionAnswering, BertTokenizer, get_home_dir

# Load the pretrained model and tokenizer
model = TFBertForQuestionAnswering.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Get the home directory for Transformers
home_dir = get_home_dir()

# Print the paths for the downloaded model and tokenizer
model_path = f"{home_dir}/.cache/torch/transformers/{model.base_model_prefix}/"
tokenizer_path = f"{home_dir}/.cache/torch/transformers/{tokenizer.name}/"

print("Model Path:", model_path)
print("Tokenizer Path:", tokenizer_path)


ImportError: cannot import name 'get_home_dir' from 'transformers' (C:\Users\user\anaconda3\lib\site-packages\transformers\__init__.py)

In [51]:
import transformers
print(transformers.__file__)


C:\Users\user\anaconda3\lib\site-packages\transformers\__init__.py


In [50]:
!pip uninstall transformers